In [ ]:
import sqlite3
import csv

def create_table():
    conn = sqlite3.connect('phonebook.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS PhoneBook (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name TEXT NOT NULL,
            phone TEXT NOT NULL UNIQUE
        )
    ''')
    conn.commit()
    conn.close()

def clear_table():
    conn = sqlite3.connect('phonebook.db')
    cursor = conn.cursor()
    cursor.execute("DELETE FROM PhoneBook")
    cursor.execute("DELETE FROM sqlite_sequence WHERE name='PhoneBook'")
    conn.commit()
    conn.close()
    print("table cleared and index reset.")

def insert_from_console():
    name = input("enter name: ")
    phone = input("enter phone number: ")
    conn = sqlite3.connect('phonebook.db')
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO PhoneBook (first_name, phone) VALUES (?, ?)", (name.strip(), phone.strip()))
        print("successfully added.")
    except sqlite3.IntegrityError:
        print("phone number already exists.")
    conn.commit()
    conn.close()

def insert_from_csv(filename):
    conn = sqlite3.connect('phonebook.db')
    cursor = conn.cursor()
    try:
        with open(filename, newline='', encoding='utf-8-sig') as csvfile:
            reader = csv.reader(csvfile)
            for i, row in enumerate(reader, 1):
                if len(row) < 2:
                    print(f"skipped line {i} (insufficient data): {row}")
                    continue
                name = row[0].replace('\ufeff', '').strip()
                phone = row[1].replace(' ', '').strip()
                if not name or not phone:
                    print(f"skipped line {i} (empty fields): {row}")
                    continue
                try:
                    cursor.execute("INSERT INTO PhoneBook (first_name, phone) VALUES (?, ?)", (name, phone))
                except sqlite3.IntegrityError:
                    print(f"skipped (duplicate): {phone}")
        print("csv import completed.")
    except FileNotFoundError:
        print("file not found.")
    conn.commit()
    conn.close()

def update_data():
    field = input("which field do you want to update? (first_name / phone): ")
    if field not in ['first_name', 'phone']:
        print("invalid field.")
        return
    old_value = input("enter current value: ")
    new_value = input("enter new value: ")

    conn = sqlite3.connect('phonebook.db')
    cursor = conn.cursor()
    cursor.execute(f"UPDATE PhoneBook SET {field} = ? WHERE {field} = ?", (new_value, old_value))
    if cursor.rowcount > 0:
        print("update successful.")
    else:
        print("no match found.")
    conn.commit()
    conn.close()

def query_data():
    filter_by = input("filter by (leave empty to show all): first_name / phone: ")
    value = None
    if filter_by in ['first_name', 'phone']:
        value = input("enter filter value: ")

    conn = sqlite3.connect('phonebook.db')
    cursor = conn.cursor()
    if filter_by and value:
        cursor.execute(f"SELECT first_name, phone FROM PhoneBook WHERE {filter_by} = ?", (value,))
    else:
        cursor.execute("SELECT first_name, phone FROM PhoneBook")
    rows = cursor.fetchall()
    if rows:
        print("\ncontact list:")
        for idx, (name, phone) in enumerate(rows, start=1):
            print(f"{idx}. {name} — {phone}")
    else:
        print("no records found.")
    conn.close()

def delete_by_value():
    field = input("delete by (first_name / phone): ")
    if field not in ['first_name', 'phone']:
        print("invalid field.")
        return
    value = input("enter value to delete: ")

    conn = sqlite3.connect('phonebook.db')
    cursor = conn.cursor()
    cursor.execute(f"DELETE FROM PhoneBook WHERE {field} = ?", (value,))
    if cursor.rowcount > 0:
        print("deletion successful.")
    else:
        print("no match found.")
    conn.commit()
    conn.close()

def menu():
    create_table()
    while True:
        print("\nphonebook menu")
        print("1. add contact (from console)")
        print("2. import contacts from csv")
        print("3. update contact")
        print("4. show contacts")
        print("5. delete contact")
        print("6. clear table")
        print("0. exit")

        choice = input("select option: ")
        if choice == "1":
            insert_from_console()
        elif choice == "2":
            filename = input("enter csv file name: ")
            insert_from_csv(filename)
        elif choice == "3":
            update_data()
        elif choice == "4":
            query_data()
        elif choice == "5":
            delete_by_value()
        elif choice == "6":
            clear_table()
        elif choice == "0":
            print("exiting...")
            break
        else:
            print("invalid choice!")

if __name__ == "__main__":
    menu()